In [33]:
import pandas as pd
from io import StringIO
import os
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Union, Set
from enum import Enum

from nst_scraper import nst_on_ice_scraper
from db_utils import update_player_db, check_last_update, append_player_ids, get_player_full_name
from team_utils import get_most_recent_game_id

from dotenv import load_dotenv

pd.set_option('display.max_columns', None)


In [34]:
player_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='')
player_stats_df.head()

,player,team,position,gp,toi,goals,total_assists,first_assists,second_assists,total_points,ipp,shots,sh%,ixg,icf,iff,iscf,ihdcf,rush_attempts,rebounds_created,pim,total_penalties,minor,major,misconduct,penalties_drawn,giveaways,takeaways,hits,hits_taken,shots_blocked,faceoffs_won,faceoffs_lost,faceoffs_%
0,Ryan Suter,STL,D,21,408.533333,1,3,1,2,4,20.00,18,5.56,0.78,51,29,11,2,3,5,12,6,6,0,0,1,22,4,12,15,23,0,0,-
1,Brent Burns,CAR,D,19,313.166667,0,5,2,3,5,29.41,31,0.00,1.25,80,50,22,2,2,6,4,2,2,0,0,1,26,8,2,12,14,0,0,-
2,Corey Perry,EDM,R,21,215.083333,4,0,0,0,4,50.00,19,21.05,2.25,41,27,24,12,0,1,14,4,2,2,0,7,6,3,7,13,5,1,2,33.33
3,Alex Ovechkin,WSH,L,18,215.250000,9,7,4,3,16,72.73,34,26.47,3.27,69,49,40,16,0,5,6,3,3,0,0,0,7,4,41,8,3,0,0,-
4,Evgeni Malkin,PIT,C,22,329.983333,3,9,6,3,12,75.00,30,10.00,4.62,62,41,36,20,0,8,10,5,5,0,0,1,21,3,11,20,16,69,88,43.95


In [35]:
goalie_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='', pos='g')
goalie_stats_df.head()

,player,team,gp,toi,shots_against,saves,goals_against,sv%,gaa,gsaa,xg_against,hd_shots_against,hd_saves,hd_goals_against,hdsv%,hdgaa,hdgsaa,md_shots_against,md_saves,md_goals_against,mdsv%,mdgaa,mdgsaa,ld_shots_against,ld_saves,ld_goals_against,ldsv%,ldgaa,ldgsaa,rush_attempts_against,rebound_attempts_against,avg._shot_distance,avg._goal_distance
0,Marc-Andre Fleury,MIN,5,250.166667,120,111,9,0.925,2.16,1.44,7.28,19,15,4,0.789,0.96,-0.56,27,23,4,0.852,0.96,-0.96,65,65,0,1.000,0.00,2.13,8,15,40.33,31.33
1,Jonathan Quick,NYR,5,225.233333,109,108,1,0.991,0.27,8.48,8.91,27,27,0,1.000,0.00,4.88,32,31,1,0.969,0.27,2.60,47,47,0,1.000,0.00,1.54,11,21,34.87,30.00
2,James Reimer,ANA,2,103.600000,53,48,5,0.906,2.90,-0.39,5.20,7,4,3,0.571,1.74,-1.73,23,21,2,0.913,1.16,0.59,23,23,0,1.000,0.00,0.75,3,8,30.81,17.20
3,Semyon Varlamov,NYI,8,405.900000,174,160,14,0.920,2.07,1.13,15.72,49,43,6,0.878,0.89,2.87,42,37,5,0.881,0.74,-0.28,67,64,3,0.955,0.44,-0.81,14,28,35.54,20.21
4,Jacob Markstrom,N.J,15,732.466667,334,304,30,0.910,2.46,-0.96,27.98,73,55,18,0.753,1.47,-4.79,91,83,8,0.912,0.66,2.23,147,143,4,0.973,0.33,0.81,44,65,39.81,20.63


In [36]:
class Position(Enum):
    C = 'C'
    L = 'L'
    R = 'R'
    D = 'D'
    G = 'G'
    
    @property
    def category(self) -> str:
        if self in {Position.C, Position.L, Position.R}:
            return 'F'
        elif self == Position.D:
            return 'D'
        elif self == Position.G:
            return 'G'
    
    def __str__(self) -> str:
        return self.value

In [37]:
@dataclass
class Player:
    name: str
    team: str
    position: Position
    player_id: Optional[int] = None

    def __str__(self) -> str:
        """
        Returns a string representation of the player.
        """
        return f"{self.name} ({self.position}) - {self.team}"

    def to_dict(self) -> Dict[str, Optional[str]]:
        """
        Converts the Player instance into a dictionary.
        
        Returns:
            Dict[str, Optional[str]]: A dictionary representation of the player.
        """
        return {
            'player_id': self.player_id,
            'name': self.name,
            'team': self.team,
            'position': self.position.value
        }

In [38]:
@dataclass
class Lineup:
    name: str
    forwards: List[Optional[Player]] = field(default_factory=lambda: [None] * 12)
    defense: List[Optional[Player]] = field(default_factory=lambda: [None] * 6)
    goalies: List[Optional[Player]] = field(default_factory=lambda: [None] * 2)
    
    ALLOWED_FORWARD_CATEGORIES = {'F'}
    ALLOWED_DEFENSE_CATEGORY = 'D'
    ALLOWED_GOALIE_CATEGORY = 'G'
    
    def __post_init__(self):
        self.validate_lineup_size()
    
    def validate_lineup_size(self):
        total_players = sum(player is not None for player in self.forwards + self.defense + self.goalies)
        if total_players > 20:
            raise ValueError(f"Total number of players ({total_players}) exceeds the hard limit of 20.")
    
    def add_player(
        self,
        category: str,
        player: Player,
        slot: int,
        allowed_categories: Union[str, Set[str]],
        max_slots: int
    ):
        """
        Adds a player to the specified category and slot after validating their position category.
        Ensures that the total number of players does not exceed 20.
        """
        if isinstance(allowed_categories, str):
            allowed_categories = {allowed_categories}
        elif isinstance(allowed_categories, set):
            allowed_categories = allowed_categories
        else:
            raise TypeError("allowed_categories must be a string or a set of strings.")
        
        if player.position.category not in allowed_categories:
            raise ValueError(
                f"Cannot add player '{player.name}' with position '{player.position.value}' "
                f"to {category}. Allowed categories: {', '.join(allowed_categories)}."
            )
        
        if not 0 <= slot < max_slots:
            raise IndexError(f"{category.capitalize()} slot must be between 0 and {max_slots - 1}.")
        
        current_category = getattr(self, category)
        if current_category[slot]:
            existing_player = current_category[slot].name
            print(f"Warning: Slot {slot + 1} in {category} is already occupied by '{existing_player}'. Overwriting.")
        
        # Check total players before adding
        total_players = sum(player is not None for player in self.forwards + self.defense + self.goalies)
        if current_category[slot] is None and total_players >= 20:
            raise ValueError("Cannot add more players. The lineup has reached the hard limit of 20 players.")
        
        current_category[slot] = player
        setattr(self, category, current_category)
        print(f"Added player '{player.name}' to {category.capitalize()} slot {slot + 1}.")
    
    def add_forward(self, player: Player, slot: int):
        self.add_player(
            category='forwards',
            player=player,
            slot=slot,
            allowed_categories=self.ALLOWED_FORWARD_CATEGORIES,
            max_slots=len(self.forwards)
        )
    
    def add_defense(self, player: Player, slot: int):
        self.add_player(
            category='defense',
            player=player,
            slot=slot,
            allowed_categories={self.ALLOWED_DEFENSE_CATEGORY},
            max_slots=len(self.defense)
        )
    
    def set_goalie(self, player: Player, slot: int):
        self.add_player(
            category='goalies',
            player=player,
            slot=slot,
            allowed_categories={self.ALLOWED_GOALIE_CATEGORY},
            max_slots=len(self.goalies)
        )
    
    def adjust_slots(self, category: str, delta: int):
        """
        Adjusts the number of slots in the specified category by delta.
        Allows ±1 adjustment only.
        
        Args:
            category (str): The category to adjust ('forwards' or 'defense').
            delta (int): The change in number of slots (+1 or -1).
        """
        if category not in {'forwards', 'defense'}:
            raise ValueError("Can only adjust 'forwards' or 'defense' categories.")
        if delta not in {-1, 1}:
            raise ValueError("Delta must be either +1 or -1.")
        
        current_slots = getattr(self, category)
        new_slot_count = len(current_slots) + delta
        
        if category == 'forwards':
            if not (11 <= new_slot_count <= 13):
                raise ValueError("Number of forwards can only vary by ±1 from the default of 12.")
        elif category == 'defense':
            if not (5 <= new_slot_count <= 7):
                raise ValueError("Number of defensemen can only vary by ±1 from the default of 6.")
        
        if delta == 1:
            current_slots.append(None)
        elif delta == -1:
            removed_player = current_slots.pop()
            if removed_player:
                print(f"Removed player '{removed_player.name}' from {category}.")
        
        setattr(self, category, current_slots)
        print(f"Adjusted {category} slots to {len(getattr(self, category))}.")
        self.validate_lineup_size()
    
    def display_lineup(self):
        """
        Prints the current lineup in a structured format.
        """
        print(f"Lineup: {self.name}\n")
        
        for category, title in [('forwards', 'Forwards'), ('defense', 'Defense'), ('goalies', 'Goalies')]:
            print(f"{title}:")
            for idx, player in enumerate(getattr(self, category), start=1):
                player_info = str(player) if player else 'Empty'
                print(f"  Slot {idx}: {player_info}")
            print()
    
    def to_dataframe(self) -> pd.DataFrame:
        """
        Converts the lineup into a pandas DataFrame.
        Conditionally includes player attributes if they are present.
        """
        data = []
        for category, pos in [('forwards', 'f'), ('defense', 'd'), ('goalies', 'g')]:
            for idx, player in enumerate(getattr(self, category), start=1):
                player_dict = {
                    'Position': f"{pos}{idx}",
                    'Player': player.name if player else 'Empty'
                }
                # Conditionally add 'player_id' if it exists
                if player and player.player_id is not None:
                    player_dict['Player ID'] = player.player_id
                data.append(player_dict)
        
        df = pd.DataFrame(data)
        
        # Optionally, remove columns where all values are NaN
        df.dropna(axis=1, how='all', inplace=True)
        
        return df
    
    def to_transposed_dataframe(self) -> pd.DataFrame:
        """
        Transposes the lineup DataFrame so that each column represents a position-slot combination
        and the row contains the corresponding player names.
        Conditionally includes additional player attributes if they are present.
        """
        df = self.to_dataframe()
        
        # Initialize dictionaries to hold player names and optional IDs
        player_data = {}
        id_data = {}
        
        for _, row in df.iterrows():
            pos = row['Position']
            player_name = row['Player']
            player_data[pos] = player_name
            
            # Handle 'Player ID' if it exists
            if 'Player ID' in row and pd.notna(row['Player ID']):
                id_data[f"{pos}_ID"] = row['Player ID']
        
        # Combine player names and IDs into a single dictionary
        transposed_data = {**player_data, **id_data}
        
        # Create the transposed DataFrame with a single row
        transposed_df = pd.DataFrame([transposed_data])
        
        return transposed_df

In [39]:
# Creating Player instances from the player_stats_df DataFrame
player_list = []
for _, row in player_stats_df.iterrows():
    try:
        position_enum = Position(row['position'])  # Convert abbreviation to Position Enum
    except ValueError:
        print(f"Invalid position '{row['position']}' for player '{row['player']}'. Skipping.")
        continue
    
    player = Player(
        name=row['player'],
        team=row['team'],
        position=position_enum
        # player_id is not set initially
    )
    player_list.append(player)

# Convert player list to DataFrame to verify
pd.DataFrame([{
    'name': player.name,
    'team': player.team, 
    'position': player.position.value
} for player in player_list]).head()

Invalid position 'C, R' for player 'Gustav Nyquist'. Skipping.
Invalid position 'C, L' for player 'Jeff Skinner'. Skipping.
Invalid position 'C, L' for player 'Mark Jankowski'. Skipping.
Invalid position 'C, R' for player 'Michael McCarron'. Skipping.
Invalid position 'C, R' for player 'Philip Tomasino'. Skipping.
Invalid position 'L, R' for player 'Cole Smith'. Skipping.


,name,team,position
0,Ryan Suter,STL,D
1,Brent Burns,CAR,D
2,Corey Perry,EDM,R
3,Alex Ovechkin,WSH,L
4,Evgeni Malkin,PIT,C


In [40]:
goalie_stats_df.head()

,player,team,gp,toi,shots_against,saves,goals_against,sv%,gaa,gsaa,xg_against,hd_shots_against,hd_saves,hd_goals_against,hdsv%,hdgaa,hdgsaa,md_shots_against,md_saves,md_goals_against,mdsv%,mdgaa,mdgsaa,ld_shots_against,ld_saves,ld_goals_against,ldsv%,ldgaa,ldgsaa,rush_attempts_against,rebound_attempts_against,avg._shot_distance,avg._goal_distance
0,Marc-Andre Fleury,MIN,5,250.166667,120,111,9,0.925,2.16,1.44,7.28,19,15,4,0.789,0.96,-0.56,27,23,4,0.852,0.96,-0.96,65,65,0,1.000,0.00,2.13,8,15,40.33,31.33
1,Jonathan Quick,NYR,5,225.233333,109,108,1,0.991,0.27,8.48,8.91,27,27,0,1.000,0.00,4.88,32,31,1,0.969,0.27,2.60,47,47,0,1.000,0.00,1.54,11,21,34.87,30.00
2,James Reimer,ANA,2,103.600000,53,48,5,0.906,2.90,-0.39,5.20,7,4,3,0.571,1.74,-1.73,23,21,2,0.913,1.16,0.59,23,23,0,1.000,0.00,0.75,3,8,30.81,17.20
3,Semyon Varlamov,NYI,8,405.900000,174,160,14,0.920,2.07,1.13,15.72,49,43,6,0.878,0.89,2.87,42,37,5,0.881,0.74,-0.28,67,64,3,0.955,0.44,-0.81,14,28,35.54,20.21
4,Jacob Markstrom,N.J,15,732.466667,334,304,30,0.910,2.46,-0.96,27.98,73,55,18,0.753,1.47,-4.79,91,83,8,0.912,0.66,2.23,147,143,4,0.973,0.33,0.81,44,65,39.81,20.63


In [41]:
# Creating Player instances from the goalie_stats_df DataFrame
goalie_list = []
for _, row in goalie_stats_df.iterrows():
    player = Player(
        name=row['player'],
        team=row['team'],
        position=Position.G  # Set position to 'G' for goalies
        # player_id is not set initially
    )
    goalie_list.append(player)

# Convert goalie list to DataFrame to verify
pd.DataFrame([{
    'name': player.name,
    'team': player.team, 
    'position': player.position.value
} for player in goalie_list]).head()

,name,team,position
0,Marc-Andre Fleury,MIN,G
1,Jonathan Quick,NYR,G
2,James Reimer,ANA,G
3,Semyon Varlamov,NYI,G
4,Jacob Markstrom,N.J,G


In [42]:
today_datetime= datetime.now()
yesterday_datetime = today_datetime - timedelta(days=1, hours=6) # UTC offset
yesterday = yesterday_datetime.strftime('%Y-%m-%d')
yesterday

'2024-11-21'

In [43]:
# Load environment variables from .env file
load_dotenv()

# Construct the database configuration dictionary
db_config = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

In [44]:
# Check the last update time of the players database
last_update = check_last_update(db_config)

INFO:db_utils:Database connection established.
INFO:db_utils:Last database update was on: 2024-11-22
INFO:db_utils:Database connection closed.


In [45]:
# Convert last_update to datetime
last_update_dt = datetime.strptime(last_update, '%Y-%m-%d')
today_dt = datetime.strptime(today_datetime.strftime('%Y-%m-%d'), '%Y-%m-%d')
yesterday_dt = datetime.strptime(yesterday, '%Y-%m-%d')

# Only update if last update was before yesterday
if last_update_dt not in [today_dt, yesterday_dt]:
    # Update the player database from last update to yesterday
    update_player_db(last_update, yesterday, db_config, skip_existing=True)
else:
    print(f"No need to update the player database. Last update was on: {last_update}")


No need to update the player database. Last update was on: 2024-11-22


In [46]:
def add_player_to_lineup(lineup: Lineup, player: Player, category: str):
    """
    Adds a player to the lineup in the specified category, handling potential errors.
    
    Args:
        lineup (Lineup): The lineup object.
        player (Player): The player to add.
        category (str): The category ('forwards', 'defense', 'goalies').
    """
    try:
        if category == 'forwards':
            slot = next(i for i, p in enumerate(lineup.forwards) if p is None)
            lineup.add_forward(player, slot)
        elif category == 'defense':
            slot = next(i for i, p in enumerate(lineup.defense) if p is None)
            lineup.add_defense(player, slot)
        elif category == 'goalies':
            slot = next(i for i, p in enumerate(lineup.goalies) if p is None)
            lineup.set_goalie(player, slot)
        else:
            print(f"Unknown category '{category}'.")
    except StopIteration:
        print(f"No available slots to add player '{player.name}' in category '{category}'.")
    except ValueError as ve:
        print(ve)
    except IndexError as ie:
        print(ie)

In [47]:
# TODO this function just creates a lineup from the player_list and goalie_list
def create_lineup(team) -> Lineup:
    """
    Creates and displays a lineup consisting of players from the specified team.
    
    Args:
        team (str): The team name to filter players.
    """
    # Creating two lineup objects
    lineup1 = Lineup("Lineup 1")
    
    # Adding forwards to lineup1
    forward_count = 0
    for player in player_list:
        if player.team == team:
            try:
                lineup1.add_forward(player, forward_count)
                forward_count += 1
                if forward_count >= 12:
                    break
            except ValueError as e:
                print(f"Skipping player '{player.name}': {e}")
            except IndexError as e:
                print(f"Skipping player '{player.name}': {e}")
        else:
            continue  # Proceed to the next player if not in the specified team
    
    # Adding defense to lineup1
    defense_count = 0
    for player in player_list:
        if player.team == team:
            try:
                lineup1.add_defense(player, defense_count)
                defense_count += 1
                if defense_count >= 6:
                    break
            except ValueError as e:
                print(f"Skipping player '{player.name}': {e}")
            except IndexError as e:
                print(f"Skipping player '{player.name}': {e}")
        else:
            continue  # Proceed to the next player if not in the specified team
    
    # Adding goalies to lineup1
    goalie_count = 0
    for goalie in goalie_list:
        if goalie.team != team:
            continue  # Proceed to the next goalie if not in the specified team
        if goalie_count >= 2:
            print("Maximum of two goalies have been assigned.")
            break
        try:
            lineup1.set_goalie(goalie, goalie_count)
            goalie_count += 1
        except ValueError as e:
            print(f"Skipping goalie '{goalie.name}': {e}")
        except IndexError as e:
            print(f"Skipping goalie '{goalie.name}': {e}")
    
    # Display the lineup
    # lineup1.display_lineup()
    return lineup1

my_lineup = create_lineup('TOR')

Added player 'Ryan Reaves' to Forwards slot 1.
Added player 'Max Pacioretty' to Forwards slot 2.
Added player 'John Tavares' to Forwards slot 3.
Skipping player 'Oliver Ekman-Larsson': Cannot add player 'Oliver Ekman-Larsson' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Chris Tanev': Cannot add player 'Chris Tanev' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Jani Hakanpää': Cannot add player 'Jani Hakanpää' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Morgan Rielly': Cannot add player 'Morgan Rielly' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Jake McCabe': Cannot add player 'Jake McCabe' with position 'D' to forwards. Allowed categories: F.
Added player 'Max Domi' to Forwards slot 4.
Added player 'William Nylander' to Forwards slot 5.
Added player 'Mitch Marner' to Forwards slot 6.
Added player 'Steven Lorentz' to Forwards slot 7.
Skipping player 'Philippe Myers': Cannot add playe

In [48]:
def create_flexible_lineup(team: str) -> Lineup:
    """
    Creates a flexible lineup for the specified team, allowing for ±1 forward or defense spot.
    
    Args:
        team (str): The team name to filter players.
    
    Returns:
        Lineup: The configured lineup object.
    """
    lineup = Lineup(f"Flexible Lineup for {team}")
    
    # Example logic to adjust slots based on team strategy
    # Here, we simply allow flexibility; implement specific rules as needed
    allow_extra_forward = True  # Example condition
    allow_extra_defense = False  # Example condition
    
    if allow_extra_forward:
        lineup.adjust_slots('forwards', 1)  # Increase forwards to 13
    elif allow_extra_defense:
        lineup.adjust_slots('defense', 1)  # Increase defense to 7
    
    # Add players to the lineup
    forward_count = 0
    defense_count = 0
    goalie_count = 0
    
    for player in player_list:
        if player.team == team:
            if player.position.category == 'F' and forward_count < len(lineup.forwards):
                try:
                    lineup.add_forward(player, forward_count)
                    forward_count += 1
                except ValueError:
                    continue
            elif player.position.category == 'D' and defense_count < len(lineup.defense):
                try:
                    lineup.add_defense(player, defense_count)
                    defense_count += 1
                except ValueError:
                    continue
    
    for goalie in goalie_list:
        if goalie.team == team and goalie_count < len(lineup.goalies):
            try:
                lineup.set_goalie(goalie, goalie_count)
                goalie_count += 1
            except ValueError:
                continue
    
    return lineup

# Create and display a flexible lineup for 'TOR'
flexible_lineup = create_flexible_lineup('TOR')
flexible_lineup.display_lineup()

Adjusted forwards slots to 13.
Added player 'Ryan Reaves' to Forwards slot 1.
Added player 'Max Pacioretty' to Forwards slot 2.
Added player 'John Tavares' to Forwards slot 3.
Added player 'Oliver Ekman-Larsson' to Defense slot 1.
Added player 'Chris Tanev' to Defense slot 2.
Added player 'Jani Hakanpää' to Defense slot 3.
Added player 'Morgan Rielly' to Defense slot 4.
Added player 'Jake McCabe' to Defense slot 5.
Added player 'Max Domi' to Forwards slot 4.
Added player 'William Nylander' to Forwards slot 5.
Added player 'Mitch Marner' to Forwards slot 6.
Added player 'Steven Lorentz' to Forwards slot 7.
Added player 'Philippe Myers' to Defense slot 6.
Added player 'Auston Matthews' to Forwards slot 8.
Added player 'David Kampf' to Forwards slot 9.
Added player 'Connor Dewar' to Forwards slot 10.
Added player 'Pontus Holmberg' to Forwards slot 11.
Added player 'Nicholas Robertson' to Forwards slot 12.
Added player 'Bobby McMann' to Forwards slot 13.
Added player 'Anthony Stolarz' to G

In [49]:
my_lineup.to_dataframe()

,Position,Player
0,f1,Ryan Reaves
1,f2,Max Pacioretty
2,f3,John Tavares
3,f4,Max Domi
4,f5,William Nylander
5,f6,Mitch Marner
6,f7,Steven Lorentz
7,f8,Auston Matthews
8,f9,David Kampf
9,f10,Connor Dewar


In [50]:
# Convert the lineup to a transposed DataFrame
transposed_lineup_df = my_lineup.to_transposed_dataframe()

# Display the transposed DataFrame
transposed_lineup_df

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,d1,d2,d3,d4,d5,d6,g1,g2
0,Ryan Reaves,Max Pacioretty,John Tavares,Max Domi,William Nylander,Mitch Marner,Steven Lorentz,Auston Matthews,David Kampf,Connor Dewar,Pontus Holmberg,Nicholas Robertson,Oliver Ekman-Larsson,Chris Tanev,Jani Hakanpää,Morgan Rielly,Jake McCabe,Philippe Myers,Anthony Stolarz,Joseph Woll


In [51]:
def assign_player_ids_to_lineup(transposed_lineup_df, db_config):
    """
    Processes the transposed_lineup_df DataFrame to append player IDs next to each player's name.
    
    This function performs the following steps:
        1. Extracts all unique player names from the lineup DataFrame.
        2. Creates Player instances for each player name.
        3. Uses the append_player_ids function to assign player IDs to each Player object.
        4. Inserts new columns into the DataFrame with the corresponding player IDs.
    
    Args:
        transposed_lineup_df (pd.DataFrame): The transposed lineup DataFrame with player names.
        db_config (dict): Database configuration with keys: dbname, user, password, host, port.
    
    Returns:
        pd.DataFrame: The updated DataFrame with player IDs added.
    """
    import pandas as pd

    # Ensure the DataFrame has only one row
    if transposed_lineup_df.shape[0] != 1:
        raise ValueError("transposed_lineup_df should contain exactly one row representing the lineup.")

    # Extract player names from the DataFrame
    player_columns = transposed_lineup_df.columns
    player_names = transposed_lineup_df.iloc[0].tolist()

    # Create Player instances
    player_list = []
    for name in player_names:
        if name != 'Empty':
            player = Player(name=name, team=None, position=None)  # Team and position can be set if available
            player_list.append(player)

    # Append player IDs using the existing function
    append_player_ids(player_list, db_config)

    # Create a mapping from player name to player_id
    name_to_id = {player.name: player.player_id for player in player_list if player.player_id is not None}

    # Initialize a copy of the DataFrame to avoid modifying the original
    updated_df = transposed_lineup_df.copy()

    # Insert player_id columns adjacent to each player name column
    for col in player_columns:
        player_name = updated_df.at[0, col]
        if player_name != 'Empty':
            player_id = name_to_id.get(player_name, None)
            id_column = f"{col}_id"
            updated_df[id_column] = player_id
        else:
            id_column = f"{col}_id"
            updated_df[id_column] = None  # Assign None for empty slots

    return updated_df

In [52]:
# Assuming you have already created `transposed_lineup_df` and `db_config`
updated_lineup_df = assign_player_ids_to_lineup(transposed_lineup_df, db_config)

# Display the updated DataFrame
updated_lineup_df

INFO:db_utils:Database connection established.
INFO:db_utils:Assigned player_id 8471817 to Ryan Reaves.
INFO:db_utils:Assigned player_id 8474157 to Max Pacioretty.
INFO:db_utils:Assigned player_id 8475166 to John Tavares.
INFO:db_utils:Assigned player_id 8477503 to Max Domi.
INFO:db_utils:Assigned player_id 8477939 to William Nylander.
INFO:db_utils:Assigned player_id 8478483 to Mitch Marner.
INFO:db_utils:Assigned player_id 8478904 to Steven Lorentz.
INFO:db_utils:Assigned player_id 8479318 to Auston Matthews.
INFO:db_utils:Assigned player_id 8480144 to David Kampf.
INFO:db_utils:Assigned player_id 8480980 to Connor Dewar.
INFO:db_utils:Assigned player_id 8480995 to Pontus Holmberg.
INFO:db_utils:Assigned player_id 8481582 to Nicholas Robertson.
INFO:db_utils:Assigned player_id 8475171 to Oliver Ekman-Larsson.
INFO:db_utils:Assigned player_id 8475690 to Chris Tanev.
INFO:db_utils:Assigned player_id 8475825 to Jani Hakanpää.
INFO:db_utils:Assigned player_id 8476853 to Morgan Rielly.
IN

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,d1,d2,d3,d4,d5,d6,g1,g2,f1_id,f2_id,f3_id,f4_id,f5_id,f6_id,f7_id,f8_id,f9_id,f10_id,f11_id,f12_id,d1_id,d2_id,d3_id,d4_id,d5_id,d6_id,g1_id,g2_id
0,Ryan Reaves,Max Pacioretty,John Tavares,Max Domi,William Nylander,Mitch Marner,Steven Lorentz,Auston Matthews,David Kampf,Connor Dewar,Pontus Holmberg,Nicholas Robertson,Oliver Ekman-Larsson,Chris Tanev,Jani Hakanpää,Morgan Rielly,Jake McCabe,Philippe Myers,Anthony Stolarz,Joseph Woll,8471817,8474157,8475166,8477503,8477939,8478483,8478904,8479318,8480144,8480980,8480995,8481582,8475171,8475690,8475825,8476853,8476931,8479026,8476932,8479361


In [53]:
# player_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='')
player_stats_df.head() 

,player,team,position,gp,toi,goals,total_assists,first_assists,second_assists,total_points,ipp,shots,sh%,ixg,icf,iff,iscf,ihdcf,rush_attempts,rebounds_created,pim,total_penalties,minor,major,misconduct,penalties_drawn,giveaways,takeaways,hits,hits_taken,shots_blocked,faceoffs_won,faceoffs_lost,faceoffs_%
0,Ryan Suter,STL,D,21,408.533333,1,3,1,2,4,20.00,18,5.56,0.78,51,29,11,2,3,5,12,6,6,0,0,1,22,4,12,15,23,0,0,-
1,Brent Burns,CAR,D,19,313.166667,0,5,2,3,5,29.41,31,0.00,1.25,80,50,22,2,2,6,4,2,2,0,0,1,26,8,2,12,14,0,0,-
2,Corey Perry,EDM,R,21,215.083333,4,0,0,0,4,50.00,19,21.05,2.25,41,27,24,12,0,1,14,4,2,2,0,7,6,3,7,13,5,1,2,33.33
3,Alex Ovechkin,WSH,L,18,215.250000,9,7,4,3,16,72.73,34,26.47,3.27,69,49,40,16,0,5,6,3,3,0,0,0,7,4,41,8,3,0,0,-
4,Evgeni Malkin,PIT,C,22,329.983333,3,9,6,3,12,75.00,30,10.00,4.62,62,41,36,20,0,8,10,5,5,0,0,1,21,3,11,20,16,69,88,43.95


In [77]:
def get_player_stats(lineup: Lineup, player_stats_df: pd.DataFrame) -> pd.DataFrame:
    """
    Gets stats for players in the lineup, maintaining lineup order.
    
    Args:
        lineup (Lineup): The lineup containing players
        player_stats_df (pd.DataFrame): DataFrame with player statistics
        
    Returns:
        pd.DataFrame: Player statistics ordered according to lineup positions
    """
    # Create ordered list of players (forwards then defense)
    players = []
    # Add forwards in order
    players.extend([p for p in lineup.forwards if p])
    # Add defense in order 
    players.extend([p for p in lineup.defense if p])
    
    # Create ordered list of player names
    player_names = [player.name for player in players]
    
    # Filter stats and reorder to match lineup order
    stats_df = player_stats_df[player_stats_df['player'].isin(player_names)]
    
    # Create ordering dictionary mapping names to their position in lineup
    name_to_position = {name: idx for idx, name in enumerate(player_names)}
    
    # Sort stats DataFrame based on lineup order and reset index
    return stats_df.assign(
        lineup_order=stats_df['player'].map(name_to_position)
    ).sort_values('lineup_order').drop('lineup_order', axis=1).reset_index(drop=True)

lineup_player_stats = get_player_stats(my_lineup, player_stats_df)

In [78]:
lineup_player_stats

,player,team,position,gp,toi,goals,total_assists,first_assists,second_assists,total_points,ipp,shots,sh%,ixg,icf,iff,iscf,ihdcf,rush_attempts,rebounds_created,pim,total_penalties,minor,major,misconduct,penalties_drawn,giveaways,takeaways,hits,hits_taken,shots_blocked,faceoffs_won,faceoffs_lost,faceoffs_%
0,Ryan Reaves,TOR,R,16,125.083333,0,1,0,1,1,50.00,7,0.00,0.26,13,10,4,0,1,0,13,5,4,1,0,4,4,2,44,7,2,0,0,-
1,Max Pacioretty,TOR,L,13,137.433333,2,4,2,2,6,85.71,21,9.52,3.05,39,34,19,13,1,2,6,3,3,0,0,2,10,2,38,21,5,2,1,66.67
2,John Tavares,TOR,C,19,249.600000,3,7,5,2,10,71.43,34,8.82,4.62,56,47,39,20,3,5,12,6,6,0,0,4,13,5,20,16,9,130,88,59.63
3,Max Domi,TOR,C,19,237.116667,0,5,4,1,5,55.56,14,0.00,0.92,33,22,13,5,1,1,21,9,8,1,0,3,13,9,6,4,11,76,82,48.10
4,William Nylander,TOR,R,20,280.333333,7,4,3,1,11,68.75,51,13.73,4.56,92,66,42,20,5,6,4,2,2,0,0,2,11,3,1,15,5,26,23,53.06
5,Mitch Marner,TOR,R,20,279.050000,2,7,3,4,9,75.00,28,7.14,2.81,55,34,31,9,1,9,2,1,1,0,0,3,18,9,9,9,8,1,2,33.33
6,Steven Lorentz,TOR,C,20,177.683333,2,1,0,1,3,60.00,16,12.50,2.53,38,26,16,8,1,3,0,0,0,0,0,3,4,6,46,16,17,4,6,40.00
7,Auston Matthews,TOR,C,13,175.916667,2,3,2,1,5,55.56,34,5.88,4.14,75,58,49,19,5,11,4,2,2,0,0,4,12,3,6,7,17,91,77,54.17
8,David Kampf,TOR,C,18,175.700000,0,2,2,0,2,50.00,13,0.00,1.11,21,17,10,7,1,2,0,0,0,0,0,2,11,3,14,16,6,53,44,54.64
9,Connor Dewar,TOR,C,4,36.550000,0,0,0,0,0,-,4,0.00,0.74,9,5,5,4,0,2,0,0,0,0,0,0,2,0,9,4,3,0,4,0.00


In [79]:
def get_goalie_stats(lineup: Lineup, goalie_stats_df: pd.DataFrame) -> pd.DataFrame:
    """
    Gets stats for goalies in the lineup, maintaining lineup order.
    
    Args:
        lineup (Lineup): The lineup containing goalies
        goalie_stats_df (pd.DataFrame): DataFrame with goalie statistics
        
    Returns:
        pd.DataFrame: Goalie statistics ordered according to lineup positions
    """
    # Create ordered list of goalies
    goalies = [goalie for goalie in lineup.goalies if goalie]
    goalie_names = [goalie.name for goalie in goalies]
    
    # Filter stats and reorder to match lineup order
    stats_df = goalie_stats_df[goalie_stats_df['player'].isin(goalie_names)]
    
    # Create ordering dictionary mapping names to their position in lineup
    name_to_position = {name: idx for idx, name in enumerate(goalie_names)}
    
    # Sort stats DataFrame based on lineup order and reset index
    return stats_df.assign(
        lineup_order=stats_df['player'].map(name_to_position)
    ).sort_values('lineup_order').drop('lineup_order', axis=1).reset_index(drop=True)

In [80]:
lineup_goalie_stats = get_goalie_stats(my_lineup, goalie_stats_df)
lineup_goalie_stats

,player,team,gp,toi,shots_against,saves,goals_against,sv%,gaa,gsaa,xg_against,hd_shots_against,hd_saves,hd_goals_against,hdsv%,hdgaa,hdgsaa,md_shots_against,md_saves,md_goals_against,mdsv%,mdgaa,mdgsaa,ld_shots_against,ld_saves,ld_goals_against,ldsv%,ldgaa,ldgsaa,rush_attempts_against,rebound_attempts_against,avg._shot_distance,avg._goal_distance
0,Anthony Stolarz,TOR,12,548.883333,267,255,12,0.955,1.31,11.22,22.5,53,46,7,0.868,0.77,2.59,68,63,5,0.926,0.55,2.65,128,128,0,1.000,0.00,4.19,31,53,39.51,18.17
1,Joseph Woll,TOR,6,294.750000,126,117,9,0.929,1.83,1.96,9.2,21,19,2,0.905,0.41,1.80,35,33,2,0.943,0.41,1.94,63,58,5,0.921,1.02,-2.94,9,23,40.42,33.44


In [58]:
yesterday

'2024-11-21'

In [59]:
get_most_recent_game_id('TOR', '2024-11-21')

2024020301

In [60]:
from game_utils import get_game_boxscore, display_boxscore

temp_data = get_game_boxscore(2024020274)

In [61]:
away_skaters, away_goalies, home_skaters, home_goalies = display_boxscore(temp_data)
away_skaters


,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,team
0,8478585,10,D. Ryan,C,0,0,0,0,0,0,0,0,1.000000,08:47,0,14,0,1,Away
1,8477406,13,M. Janmark,C,0,1,1,1,0,1,0,0,0.000000,14:05,2,18,0,0,Away
2,8475786,18,Z. Hyman,L,0,1,1,-1,2,0,0,2,0.000000,21:52,0,22,1,0,Away
3,8474641,19,A. Henrique,C,1,0,1,1,0,0,0,2,0.777778,16:33,1,22,0,0,Away
4,8477015,28,C. Brown,R,0,1,1,1,0,1,0,1,0.000000,13:43,1,20,0,0,Away
5,8477934,29,L. Draisaitl,C,1,1,2,0,0,0,0,3,0.545455,22:09,1,23,0,0,Away
6,8475784,53,J. Skinner,L,0,0,0,-1,0,0,0,1,0.000000,11:10,1,15,1,0,Away
7,8470621,90,C. Perry,R,0,0,0,-1,0,0,0,1,1.000000,10:51,0,14,0,0,Away
8,8481617,92,V. Podkolzin,R,0,0,0,0,0,4,0,2,0.000000,12:10,1,16,0,0,Away
9,8476454,93,R. Nugent-Hopkins,C,0,1,1,-1,0,0,0,0,0.000000,20:57,1,21,1,0,Away


In [62]:
away_goalies

,playerId,sweaterNumber,name,position,evenStrengthShotsAgainst,powerPlayShotsAgainst,shorthandedShotsAgainst,saveShotsAgainst,evenStrengthGoalsAgainst,powerPlayGoalsAgainst,shorthandedGoalsAgainst,pim,goalsAgainst,toi,starter,shotsAgainst,saves,team,savePctg,decision
0,8475717,30,C. Pickard,G,0/0,0/0,0/0,0/0,0,0,0,0,0,00:00,False,0,0,Away,NaN,NaN
1,8479973,74,S. Skinner,G,12/16,4/4,2/2,18/22,4,0,0,0,4,60:04,True,22,18,Away,0.818182,O


In [63]:
home_skaters

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,team
0,8477503,11,M. Domi,C,0,0,0,0,0,0,0,1,0.333333,14:42,1,20,2,1,Home
1,8478483,16,M. Marner,R,1,1,2,3,0,0,0,3,0.000000,24:00,0,25,0,1,Home
2,8478904,18,S. Lorentz,C,0,0,0,0,0,0,0,0,0.000000,06:46,0,11,0,0,Home
3,8482720,23,M. Knies,L,1,0,1,0,0,1,0,1,0.000000,21:05,0,23,0,0,Home
4,8480980,24,C. Dewar,C,0,0,0,0,0,4,0,0,0.000000,08:40,0,14,1,0,Home
5,8480995,29,P. Holmberg,R,0,0,0,-1,0,0,0,0,0.000000,13:02,2,16,0,0,Home
6,8480144,64,D. Kampf,C,0,0,0,0,0,1,0,0,0.400000,14:22,1,19,1,0,Home
7,8482259,74,B. McMann,C,2,0,2,2,0,1,0,3,0.000000,19:06,2,23,0,0,Home
8,8471817,75,R. Reaves,R,0,0,0,0,15,1,0,0,0.000000,03:54,0,5,0,0,Home
9,8477939,88,W. Nylander,R,0,0,0,-1,0,0,0,4,0.571429,21:18,0,21,0,0,Home


In [64]:
home_goalies

,playerId,sweaterNumber,name,position,evenStrengthShotsAgainst,powerPlayShotsAgainst,shorthandedShotsAgainst,saveShotsAgainst,savePctg,evenStrengthGoalsAgainst,powerPlayGoalsAgainst,shorthandedGoalsAgainst,pim,goalsAgainst,toi,starter,decision,shotsAgainst,saves,team
0,8476932,41,A. Stolarz,G,24/26,2/3,1/1,27/30,0.9,2,1,0,0,3,60:40,True,W,30,27,Home
1,8479361,60,J. Woll,G,0/0,0/0,0/0,0/0,NaN,0,0,0,0,0,00:00,False,NaN,0,0,Home


In [71]:
def extract_team_lineup(team: str, reference_date: Optional[str] = None) -> Lineup:
    """
    Extracts the most recent lineup for the specified team based on the latest game data.

    This function performs the following steps:
        1. Determines the reference date (defaults to yesterday if not provided).
        2. Retrieves the most recent game ID for the team using `get_most_recent_game_id`.
        3. Fetches the game boxscore data using `get_game_boxscore`.
        4. Processes the boxscore to obtain skaters and goalies using `display_boxscore`.
        5. Constructs and returns a `Lineup` object populated with the team's players.

    Args:
        team (str): The three-letter team code (e.g., 'TOR').
        reference_date (Optional[str]): The reference date in 'YYYY-MM-DD' format. Defaults to yesterday's date.

    Returns:
        Lineup: A `Lineup` object containing the team's players from the most recent game.

    Raises:
        ValueError: If no recent game is found for the team or if the team is not part of the retrieved game.
    """
    # Step 1: Determine the reference date
    if reference_date is None:
        today_datetime = datetime.now()
        yesterday_datetime = today_datetime - timedelta(days=1, hours=6)  # Adjust for UTC offset if necessary
        reference_date = yesterday_datetime.strftime('%Y-%m-%d')

    # Step 2: Retrieve the most recent game ID for the team
    game_id = get_most_recent_game_id(team, reference_date)
    if game_id is None:
        raise ValueError(f"No recent game found for team '{team}' before {reference_date}.")

    # Print the game_id
    print(f"Game ID: {game_id}")

    # Step 3: Fetch the game boxscore data
    game_data = get_game_boxscore(game_id)

    # Step 4: Process the boxscore to obtain skaters and goalies
    away_skaters, away_goalies, home_skaters, home_goalies = display_boxscore(game_data)

    # Extract team abbrevs to determine if the team is home or away
    away_team_code = game_data.get('awayTeam', {}).get('abbrev')
    home_team_code = game_data.get('homeTeam', {}).get('abbrev')

    if not away_team_code or not home_team_code:
        raise ValueError("Team abbreviations not found in game data.")

    if team.upper() == away_team_code.upper():
        team_side = 'Away'
        skaters = away_skaters
        goalies = away_goalies
    elif team.upper() == home_team_code.upper():
        team_side = 'Home'
        skaters = home_skaters
        goalies = home_goalies
    else:
        raise ValueError(f"Team '{team}' not found in game ID {game_id}.")

    # Step 5: Construct the Lineup object
    lineup = Lineup(name=f"{team.upper()} Lineup from Game {game_id}")

    # Add Skaters to the Lineup
    for _, skater in skaters.iterrows():
        try:
            position_enum = Position(skater['position'])  # Convert to Position Enum
        except ValueError:
            print(f"Invalid position '{skater['position']}' for player '{skater['name']}'. Skipping.")
            continue

        player = Player(
            player_id=skater['playerId'],
            name=get_player_full_name(skater['playerId'], db_config),
            team=team.upper(),
            position=position_enum
        )

        # Add player to the appropriate category in the lineup
        if player.position.category == 'F':
            try:
                empty_slot = next(i for i, p in enumerate(lineup.forwards) if p is None)
                lineup.add_forward(player, empty_slot)
            except StopIteration:
                print(f"No available forward slot to add player '{player.name}'.")
        elif player.position.category == 'D':
            try:
                empty_slot = next(i for i, p in enumerate(lineup.defense) if p is None)
                lineup.add_defense(player, empty_slot)
            except StopIteration:
                print(f"No available defense slot to add player '{player.name}'.")
        else:
            print(f"Player '{player.name}' has an unrecognized category '{player.position.category}'. Skipping.")

    # Add Goalies to the Lineup
    for _, goalie in goalies.iterrows():
        player = Player(
            player_id=goalie['playerId'],
            name=get_player_full_name(goalie['playerId'], db_config),
            team=team.upper(),
            position=Position.G
        )
        try:
            empty_slot = next(i for i, p in enumerate(lineup.goalies) if p is None)
            lineup.set_goalie(player, empty_slot)
        except StopIteration:
            print(f"No available goalie slot to add player '{player.name}'.")

    return lineup

In [73]:
toronto_lineup = extract_team_lineup('PHI', '2024-11-15')
toronto_lineup.to_transposed_dataframe()

Game ID: 2024020258


INFO:db_utils:Database connection established.
INFO:db_utils:Retrieved full name 'Bobby Brink' for player_id 8481553.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Retrieved full name 'Travis Konecny' for player_id 8478439.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Retrieved full name 'Garnet Hathaway' for player_id 8477903.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Retrieved full name 'Scott Laughton' for player_id 8476872.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Retrieved full name 'Ryan Poehling' for player_id 8480068.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Retrieved full name 'Noah Cates' for player_id 8480220.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connec

Added player 'Bobby Brink' to Forwards slot 1.
Added player 'Travis Konecny' to Forwards slot 2.
Added player 'Garnet Hathaway' to Forwards slot 3.
Added player 'Scott Laughton' to Forwards slot 4.
Added player 'Ryan Poehling' to Forwards slot 5.
Added player 'Noah Cates' to Forwards slot 6.
Added player 'Matvei Michkov' to Forwards slot 7.
Added player 'Morgan Frost' to Forwards slot 8.


INFO:db_utils:Database connection established.
INFO:db_utils:Retrieved full name 'Tyson Foerster' for player_id 8482159.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Retrieved full name 'Owen Tippett' for player_id 8480015.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Retrieved full name 'Joel Farabee' for player_id 8480797.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Retrieved full name 'Anthony Richard' for player_id 8478409.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Retrieved full name 'Egor Zamula' for player_id 8481178.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Retrieved full name 'Travis Sanheim' for player_id 8477948.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database conne

Added player 'Tyson Foerster' to Forwards slot 9.
Added player 'Owen Tippett' to Forwards slot 10.
Added player 'Joel Farabee' to Forwards slot 11.
Added player 'Anthony Richard' to Forwards slot 12.
Added player 'Egor Zamula' to Defense slot 1.
Added player 'Travis Sanheim' to Defense slot 2.
Added player 'Nick Seeler' to Defense slot 3.
Added player 'Emil Andrae' to Defense slot 4.
Added player 'Rasmus Ristolainen' to Defense slot 5.


INFO:db_utils:Database connection established.
INFO:db_utils:Retrieved full name 'Erik Johnson' for player_id 8473446.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Retrieved full name 'Aleksei Kolosov' for player_id 8482783.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Retrieved full name 'Ivan Fedotov' for player_id 8478905.
INFO:db_utils:Database connection closed.


Added player 'Erik Johnson' to Defense slot 6.
Added player 'Aleksei Kolosov' to Goalies slot 1.
Added player 'Ivan Fedotov' to Goalies slot 2.


,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,d1,d2,d3,d4,d5,d6,g1,g2,f1_ID,f2_ID,f3_ID,f4_ID,f5_ID,f6_ID,f7_ID,f8_ID,f9_ID,f10_ID,f11_ID,f12_ID,d1_ID,d2_ID,d3_ID,d4_ID,d5_ID,d6_ID,g1_ID,g2_ID
0,Bobby Brink,Travis Konecny,Garnet Hathaway,Scott Laughton,Ryan Poehling,Noah Cates,Matvei Michkov,Morgan Frost,Tyson Foerster,Owen Tippett,Joel Farabee,Anthony Richard,Egor Zamula,Travis Sanheim,Nick Seeler,Emil Andrae,Rasmus Ristolainen,Erik Johnson,Aleksei Kolosov,Ivan Fedotov,8481553,8478439,8477903,8476872,8480068,8480220,8484387,8480028,8482159,8480015,8480797,8478409,8481178,8477948,8476372,8482126,8477499,8473446,8482783,8478905


In [74]:
lineup_player_stats = get_player_stats(toronto_lineup, player_stats_df)
lineup_player_stats


,player,team,position,gp,toi,goals,total_assists,first_assists,second_assists,total_points,ipp,shots,sh%,ixg,icf,iff,iscf,ihdcf,rush_attempts,rebounds_created,pim,total_penalties,minor,major,misconduct,penalties_drawn,giveaways,takeaways,hits,hits_taken,shots_blocked,faceoffs_won,faceoffs_lost,faceoffs_%
12,Erik Johnson,PHI,D,15,168.366667,1,0,0,0,1,16.67,5,20.00,0.33,17,10,7,0,0,3,9,3,2,1,0,2,10,5,22,18,10,0,0,-
117,Nick Seeler,PHI,D,15,224.550000,1,5,1,4,6,66.67,12,8.33,0.28,35,19,7,1,1,4,9,3,2,1,0,1,18,5,14,16,31,0,0,-
159,Scott Laughton,PHI,C,20,234.916667,2,4,3,1,6,75.00,26,7.69,2.37,46,38,24,12,3,4,6,3,3,0,0,2,18,4,57,8,10,57,58,49.57
236,Rasmus Ristolainen,PHI,D,20,327.600000,1,1,1,0,2,22.22,19,5.26,0.72,34,24,7,2,2,1,4,2,2,0,0,5,18,7,32,31,21,0,0,-
253,Garnet Hathaway,PHI,R,20,192.850000,2,5,1,4,7,87.50,22,9.09,1.91,40,34,22,10,1,4,18,9,9,0,0,10,9,1,52,16,16,2,4,33.33
267,Travis Sanheim,PHI,D,20,390.583333,4,3,1,2,7,41.18,39,10.26,2.53,82,57,30,6,7,5,10,5,5,0,0,5,22,17,14,32,31,0,0,-
311,Anthony Richard,PHI,C,7,69.350000,1,3,3,0,4,66.67,7,14.29,0.52,12,11,7,2,0,2,0,0,0,0,0,0,1,2,4,5,7,0,0,-
323,Travis Konecny,PHI,R,20,292.116667,2,9,5,4,11,100.00,29,6.90,2.94,58,44,34,12,3,4,23,10,9,1,0,11,21,5,23,17,10,7,11,38.89
447,Owen Tippett,PHI,R,20,261.900000,4,0,0,0,4,50.00,40,10.00,3.32,94,65,40,10,4,6,2,1,1,0,0,2,12,9,32,20,14,4,4,50.00
451,Morgan Frost,PHI,C,16,183.633333,1,1,0,1,2,50.00,20,5.00,2.15,34,27,22,12,2,2,8,4,4,0,0,2,8,4,16,6,14,68,71,48.92


In [39]:
lineup_goalie_stats = get_goalie_stats(toronto_lineup, goalie_stats_df)
lineup_goalie_stats

,player,team,gp,toi,shots_against,saves,goals_against,sv%,gaa,gsaa,xg_against,hd_shots_against,hd_saves,hd_goals_against,hdsv%,hdgaa,hdgsaa,md_shots_against,md_saves,md_goals_against,mdsv%,mdgaa,mdgsaa,ld_shots_against,ld_saves,ld_goals_against,ldsv%,ldgaa,ldgsaa,rush_attempts_against,rebound_attempts_against,avg._shot_distance,avg._goal_distance
19,Anthony Stolarz,TOR,12,548.883333,267,255,12,0.955,1.31,11.24,22.5,53,46,7,0.868,0.77,2.58,68,63,5,0.926,0.55,2.66,128,128,0,1.000,0.00,4.20,31,53,39.51,18.17
47,Joseph Woll,TOR,6,294.750000,126,117,9,0.929,1.83,1.97,9.2,21,19,2,0.905,0.41,1.79,35,33,2,0.943,0.41,1.94,63,58,5,0.921,1.02,-2.93,9,23,40.42,33.44
